In [ ]:
import os
import shutil
os.chdir('/content')
! git clone --branch errors https://github.com/mwl10/hetvae
os.chdir('/content/hetvae')
! pip install -r requirements.txt
os.chdir('/content/hetvae/src')
import numpy as np
import torch
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import models
from argparse import Namespace
import torch.optim as optim
import utils
import my_utils
import pandas as pd
import importlib
import vae_models
import matplotlib.pyplot as plt
import sys
from dataset import DataSet
%matplotlib inline

Cloning into 'hetvae'...
remote: Enumerating objects: 2117, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 2117 (delta 176), reused 226 (delta 156), pack-reused 1860
Receiving objects: 100% (2117/2117), 26.67 MiB | 30.62 MiB/s, done.
Resolving deltas: 100% (779/779), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Pick your poison:

In [ ]:
#------------------------------
# AGN Preprocessing, we have a DF w/ OBJECT , FILTER, MJD, FLUX, ERROR
#------------------------------

def get_AGN_data(file):
    agn_df = pd.read_csv(file)
    # handle formatting for new AGN
    lcs = []
    for lc_df in agn_df.groupby('Filter'):
        lc = lc_df[1][['MJD', 'Flux', 'Error']].to_numpy()
        lcs.append(lc)

    AGN_1H2106 = DataSet()
    AGN_1H2106.dataset = lcs # skip add files part
    AGN_1H2106 = AGN_1H2106.handle_dups().prune_outliers().resample_dataset(num_samples=1).normalize().reorder() \
        .set_union_x().zero_fill().error_to_sample_weight().make_masks(frac=0.5)

    print(
        AGN_1H2106.union_x.shape,
        AGN_1H2106.dataset.shape,
        AGN_1H2106.subsampled_mask.shape,
        AGN_1H2106.recon_mask.shape
      )

    return AGN_1H2106

AGN_1H2106 = get_AGN_data('/content/hetvae/data/AGN_1H2106-099/1H2106-099_latest_lcs_pyroa.csv')

indexes of outliers to be pruned, if any: [600 601]
indexes of outliers to be pruned, if any: []
indexes of outliers to be pruned, if any: [418 583]
indexes of outliers to be pruned, if any: []
indexes of outliers to be pruned, if any: [ 98 205 260 261 554 555]
generating 1 new sample of each example in the dataset & appending them 
 old dataset length: 5
new dataset length: 10
created union_x attribute of length 2436
zero fill all the examples up to the length of longest one given, dataset is also now a numpy array w shape: (10, 619, 3), instead of a list of numpy arrays
created subsampled_mask & recon_mask instance attributes
(2436,) (10, 619, 3) (10, 619) (10, 619)


/content/hetvae/src/dataset.py:153: RuntimeWarning:

divide by zero encountered in true_divide




## Setting up arguments

In [ ]:
def define_model_args(trial):

    args = Namespace(
        batch_size = trial.suggest_categorical("batch_size", [8,16,32]),
        bound_variance = True,
        const_var = False,
        dataset='toy',
        dropout = trial.suggest_float("dropout", 0.0,0.5),
        elbo_weight = trial.suggest_float("elbo_weight", 0.0, 5.0),
        embed_time = trial.suggest_categorical("embed_time", [8,16,32,64,128]),
        enc_num_heads=4,#trial.suggest_categorical("enc_num_heads", [1,2,4,8,16]),
        intensity=True,
        k_iwae=1,
        kl_annealing=False,#trial.suggest_categorical("kl_annealing",False),
        kl_zero=False, 
        latent_dim=trial.suggest_categorical("latent_dim", [8,16,32,64,128]),
        lr=trial.suggest_float("lr", 1e-7, 1e-1, log=True),
        mixing=trial.suggest_categorical("mixing", ["concat", "concat_and_mix"]),#"separate", "interp_only", "na"]),
        mse_weight=trial.suggest_float("mse_weight",1,10),
        n=trial.suggest_categorical("n", [8,16,32,64,128]),
        net='hetvae', 
        niters=1000, 
        norm=True, 
        normalize_input='znorm', 
        num_ref_points=trial.suggest_categorical("num_ref_points", [8,16,32,64,128]),
        rec_hidden=trial.suggest_categorical("rec_hidden", [8,16,32,64,128]),
        recon_loss=False, 
        sample_tp= trial.suggest_float("sample_tp", 0.1,0.9), # will be ignored
        save=True, 
        seed=0, 
        shuffle=True, 
        std=0.1, 
        var_per_dim=False, 
        width=trial.suggest_categorical("width", [8,16,32,64,128])
    )

    return args



In [ ]:
# FILES = glob('/content/hetvae/data/EDELSON/*')[:4]
# Edelson = DataSet().add_files(FILES).files_to_numpy().handle_dups().prune_outliers().resample_dataset(num_samples=1) \
#         .normalize().set_union_x().zero_fill().make_masks(frac=0.5)

# plt.scatter(Edelson.dataset[0,:-100,0], Edelson.dataset[0,:-100,1])
        

dims of /content/hetvae/data/EDELSON/NGC5548_SX.dat:	(267, 3)
dims of /content/hetvae/data/EDELSON/NGC5548_W1.dat:	(260, 3)
dims of /content/hetvae/data/EDELSON/NGC4593_W2.dat:	(147, 3)
dims of /content/hetvae/data/EDELSON/NGC5548_M2.dat:	(248, 3)
indexes of outliers to be pruned, if any: [ 87  96 102]
indexes of outliers to be pruned, if any: []
indexes of outliers to be pruned, if any: []
indexes of outliers to be pruned, if any: []
generating 1 new sample of each example in the dataset & appending them 
 old dataset length: 4
new dataset length: 8
created union_x attribute of length 916
zero fill all the examples up to the length of longest one given, dataset is also now a numpy array w shape: (8, 264, 3), instead of a list of numpy arrays
created subsampled_mask & recon_mask instance attributes


## **Generating Synthetic Data**

## **Loading HeTVAE model**

In [ ]:
EPOCHS = 100
FILES = glob('/content/hetvae/data/EDELSON/*')[:4]
#FILES = glob('/content/hetvae/data/CAT/*/*')[:50]


DIM = 1
# definitly want a decaying error rate...
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[], gamma=0.1)


In [ ]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
def objective(trial):

    args = define_model_args(trial)

    seed = args.seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # dataset hyperparams
    num_samples=trial.suggest_int("num_samples", 1,10)

    with suppress_stdout():
        Edelson = DataSet().add_files(FILES).files_to_numpy().handle_dups().prune_outliers().resample_dataset(num_samples=num_samples) \
            .normalize().set_union_x().zero_fill().make_masks(frac=args.sample_tp)
    
    LightCurves = np.concatenate((Edelson.dataset, Edelson.subsampled_mask[:,:,np.newaxis], Edelson.recon_mask[:,:,np.newaxis]), axis=-1) # format the masks for the model 

    training, valid = np.split(LightCurves, [int(np.floor(0.8*len(LightCurves)))])# shuffle?
    
    
    train_loader = torch.utils.data.DataLoader(training, batch_size=args.batch_size)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=args.batch_size)
    
    
    net = models.load_network(args, DIM, torch.Tensor(Edelson.union_x)) # , device="cuda"
    

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam","RMSprop", "SGD"])
    optimizer = getattr(optim, optimizer_name)(net.parameters(), lr=args.lr)

    for epoch in range(EPOCHS):
        train(net, optimizer, epoch, train_loader, args, device=device)
        nll_loss = my_utils.evaluate(net, valid_loader, device=device)
        trial.report(nll_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return nll_loss


In [ ]:
study = optuna.create_study(direction="minimize")

[I 2022-06-10 03:08:22,664] A new study created in memory with name: no-name-7e1bf650-285d-419a-bbb2-fb1270169548


In [ ]:

study.optimize(objective, n_trials=10, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Iter: 0, train loss: 14.4280, avg nll: 1.5245, avg kl: 3.7352, mse: 1.001322, mae: 0.814122


[I 2022-06-10 03:09:01,942] Trial 1 finished with value: 1.5205597877502441 and parameters: {'batch_size': 16, 'dropout': 0.3584898446379959, 'elbo_weight': 1.9500108393082172, 'embed_time': 8, 'latent_dim': 32, 'lr': 3.790321359941624e-05, 'mixing': 'concat_and_mix', 'mse_weight': 4.165954942901848, 'n': 32, 'num_ref_points': 128, 'rec_hidden': 128, 'sample_tp': 0.8718641180657247, 'width': 64, 'num_samples': 5, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 1.0672093629837036.


Iter: 0, train loss: 14.4490, avg nll: 1.7028, avg kl: 0.6470, mse: 1.040388, mae: 0.834601


[I 2022-06-10 03:09:08,445] Trial 2 finished with value: 1.4265978336334229 and parameters: {'batch_size': 8, 'dropout': 0.4870411714356534, 'elbo_weight': 2.3280819962947974, 'embed_time': 64, 'latent_dim': 16, 'lr': 7.051788610000135e-05, 'mixing': 'concat_and_mix', 'mse_weight': 8.629816449479689, 'n': 8, 'num_ref_points': 64, 'rec_hidden': 16, 'sample_tp': 0.6354847642839492, 'width': 8, 'num_samples': 8, 'optimizer': 'Adam'}. Best is trial 0 with value: 1.0672093629837036.


Iter: 0, train loss: 12.6774, avg nll: 1.6251, avg kl: 0.1879, mse: 1.024311, mae: 0.826675


...

Iter: 0, train loss: 10.9017, avg nll: 1.5528, avg kl: 0.3077, mse: 0.957809, mae: 0.801380


[I 2022-06-10 03:09:22,617] Trial 6 pruned. 
[I 2022-06-10 03:09:22,775] Trial 7 pruned. 


Iter: 0, train loss: 14.1818, avg nll: 1.6084, avg kl: 1.0896, mse: 1.001631, mae: 0.814947
Iter: 0, train loss: 10.0699, avg nll: 1.5266, avg kl: 1.0181, mse: 1.015380, mae: 0.826598


[I 2022-06-10 03:09:33,736] Trial 8 pruned. 
[I 2022-06-10 03:09:33,770] Trial 9 pruned. 
/usr/local/lib/python3.7/dist-packages/optuna/pruners/_percentile.py:21: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(values)
[I 2022-06-10 03:09:33,884] Trial 10 pruned. 


Iter: 0, train loss: 8.3588, avg nll: 1.7227, avg kl: 0.2539, mse: 1.050212, mae: 0.837834
Iter: 0, train loss: nan, avg nll: nan, avg kl: nan, mse: nan, mae: nan
Study statistics: 
  Number of finished trials:  11
  Number of pruned trials:  6
  Number of complete trials:  5
Best trial:
  Value:  1.0672093629837036
  Params: 
    batch_size: 16
    dropout: 0.13813653865785686
    elbo_weight: 0.9760706051679247
    embed_time: 32
    latent_dim: 128
    lr: 0.000627218335170129
    mixing: concat_and_mix
    mse_weight: 4.005494040668514
    n: 128
    num_ref_points: 64
    rec_hidden: 128
    sample_tp: 0.4007998873024089
    width: 128
    num_samples: 6
    optimizer: RMSprop


## **Training**

In [ ]:
def train(net, optimizer,epoch, train_loader, args, device="cuda"):
      
      train_loss = 0.
      train_n = 0.
      avg_loglik, avg_kl, mse, mae = 0., 0., 0., 0.
      for i, train_batch in enumerate(train_loader):
          batch_len = train_batch.shape[0] 
          train_batch = train_batch.to(device)
          x = train_batch[:,:,0]
          y = train_batch[:,:,1:2]
          
          subsampled_mask = train_batch[:,:,3:4]
          recon_mask = train_batch[:,:,4:]
          sample_weight = train_batch[:,:,2:3]
          seqlen = train_batch.size(1) 
          # subsampled flux values and their corresponding masks....
          context_y = torch.cat((
              y * subsampled_mask, subsampled_mask
          ), -1) 
          recon_context_y = torch.cat((            # flux values with only recon_mask values showing
                  y * recon_mask, recon_mask
              ), -1)
          
    # #   def compute_unsupervised_loss(self, context_x, context_y, target_x, target_y, num_samples=1, beta=1):
          loss_info = net.compute_unsupervised_loss(
              x, # context_x, times
              context_y,           
              x, # target_x, same times, can project to arbitrary times 
              recon_context_y,
              num_samples=args.k_iwae,
              beta=1,
              #sample_weight = sample_weight   # default is 1. (no errors provided)

          )
          optimizer.zero_grad()
          loss_info.composite_loss.backward()
          optimizer.step()
          #scheduler.step()
          train_loss += loss_info.composite_loss.item() * batch_len
          avg_loglik += loss_info.loglik * batch_len
          avg_kl += loss_info.kl * batch_len
          mse += loss_info.mse * batch_len
          mae += loss_info.mae * batch_len
          train_n += batch_len
      
      
      if epoch % 100 == 0:
          print(
              'Iter: {}, train loss: {:.4f}, avg nll: {:.4f}, avg kl: {:.4f}, '
              'mse: {:.6f}, mae: {:.6f}'.format(
                  epoch,
                  train_loss / train_n,
                  -avg_loglik / train_n,
                  avg_kl / train_n,
                  mse / train_n,
                  mae / train_n
              )
          )
      
        


In [ ]:
my_utils.viz_per_example(Edelson.dataset[1], net, device=device, k_iwae=10, n_max=10, fracs=[0.05,0.1,0.15])

NameError: ignored

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study, params=["optimizer"])